اليوم سنستعمل المتجهات الموقعية للتنبؤ بالقسم الكلامي الذي تنتمي له الكلمة. و *المتجهه الموقعي* يرمّز السياق في الجملة، فيما ترمّز المتجهات الأخرى التي قد استعملناها خصائص لغوية محددة بغض النظر عن مواقعها. ونبدأ كالمعتاد بتحميل متطلبات العمل.

In [1]:
from text_analytics import TextAnalytics
import os
import pandas as pd

ai = TextAnalytics()
ai.data_dir = os.path.join("..", "data")
print("Done!")

Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.


Done!


وهذه المرة سنعمل على مدونات من مشروع التبعيات العالمي الشجري Universal Dependencies project project (universaldependencies.org). وقد عدلنا على المدونات الأصلية، ولكن التنسيق بقي مشابها جدا. فكل كلمة عبارة عن صف، ويمثل شكل الكلمة عمودا واحدا، ووسم القسم الكلامي عمودا ثانيا. ولدينا أيضا معلومات تتعلق بالجملة. وهذا يدفعنا للتأكد من أننا لم نرمز سلاسل الكلمات التي تتجاوز حدود الجملة.

In [2]:
file = os.path.join(ai.data_dir, "syntax.pos_english.gz")
df = pd.read_csv(file)
print(df)

        Source  Sentence_ID Word_ID    Word    POS
0       en_ewt            1       1    from    ADP
1       en_ewt            1       2     the    DET
2       en_ewt            1       3      ap  PROPN
3       en_ewt            1       4   comes   VERB
4       en_ewt            1       5    this    DET
...        ...          ...     ...     ...    ...
559457  en_pud        32356      21       a    DET
559458  en_pud        32356      22  friend   NOUN
559459  en_pud        32356      23      of    ADP
559460  en_pud        32356      24   peace   NOUN
559461  en_pud        32356      25       .  PUNCT

[559462 rows x 5 columns]


ولننشئ الآن إطارا للبيانات وتجربة جملة واحدة فقط.

In [3]:
test_df = df.loc[df.loc[:,"Sentence_ID"]==2]
print(test_df)

    Source  Sentence_ID Word_ID         Word    POS
7   en_ewt            2       1    president  PROPN
8   en_ewt            2       2         bush  PROPN
9   en_ewt            2       3           on    ADP
10  en_ewt            2       4      tuesday  PROPN
11  en_ewt            2       5    nominated   VERB
12  en_ewt            2       6          two    NUM
13  en_ewt            2       7  individuals   NOUN
14  en_ewt            2       8           to   PART
15  en_ewt            2       9      replace   VERB
16  en_ewt            2      10     retiring   VERB
17  en_ewt            2      11      jurists   NOUN
18  en_ewt            2      12           on    ADP
19  en_ewt            2      13      federal    ADJ
20  en_ewt            2      14       courts   NOUN
21  en_ewt            2      15           in    ADP
22  en_ewt            2      16          the    DET
23  en_ewt            2      17   washington  PROPN
24  en_ewt            2      18         area   NOUN
25  en_ewt  

وهذا الكود أدناه سيكرر على كل كلمة في الجملة ويولد متجها موقعيا لتلك الكلمة المحددة متضمنا ذلك المتجه كلمتين سابقتين وكلمتين تاليتين. ثم نطبع المتجه ونحفظه.

In [4]:
x_vectors = []
y_vector = []

#A list of words and a list of ground-truth labels
words = test_df.loc[:,"Word"].values
tags = test_df.loc[:,"POS"].values
            
#Create a positional vector for each word in the sentence
for i in range(len(words)):
    y_vector.append(tags[i])
    vector = []
                
    #Find the correct context window, filling in slots at the edges
    for j in [-2, -1, 0, 1, 2]:
        if i+j < 0 or i+j > len(words)-1:
            vector.append("#")
        else:
            vector.append(words[i+j])
                        
    #Save the positional vector for this word
    print(vector)
    x_vectors.append(vector)

['#', '#', 'president', 'bush', 'on']
['#', 'president', 'bush', 'on', 'tuesday']
['president', 'bush', 'on', 'tuesday', 'nominated']
['bush', 'on', 'tuesday', 'nominated', 'two']
['on', 'tuesday', 'nominated', 'two', 'individuals']
['tuesday', 'nominated', 'two', 'individuals', 'to']
['nominated', 'two', 'individuals', 'to', 'replace']
['two', 'individuals', 'to', 'replace', 'retiring']
['individuals', 'to', 'replace', 'retiring', 'jurists']
['to', 'replace', 'retiring', 'jurists', 'on']
['replace', 'retiring', 'jurists', 'on', 'federal']
['retiring', 'jurists', 'on', 'federal', 'courts']
['jurists', 'on', 'federal', 'courts', 'in']
['on', 'federal', 'courts', 'in', 'the']
['federal', 'courts', 'in', 'the', 'washington']
['courts', 'in', 'the', 'washington', 'area']
['in', 'the', 'washington', 'area', '.']
['the', 'washington', 'area', '.', '#']
['washington', 'area', '.', '#', '#']


إن تلك السلاسل من الكلمات هي مجرد مصفوفات من السلاسل الحرفية. فلا بد من تحويل ذلك إلى خط الترميز الأحادي one-hot encoding حيث يكون لكل موضع فيه عمود فريد في المتجه. وهنا نحول تلك السلاسل الحرفية إلى خطوط الترميزات الأحادية، ثم نطبع المصفوفات كاملة لنرى كيف تبدو. إن كل صف (كل مصفوفة جديدة) يمثل كلمة واحدة فقط، والسياق المحيط بها.

In [5]:
import numpy as np
np.set_printoptions(threshold=np.inf)
from sklearn.preprocessing import OneHotEncoder

#With all sentences finished, conert into numpy array
x_vectors = np.array(x_vectors)
y_vector = np.array(y_vector)

#Convert into a one-hot encoding
encoder = OneHotEncoder(categories='auto', handle_unknown='ignore')
encoder.fit(x_vectors)
        
#Return the x and y vectors
x_vectors = encoder.transform(x_vectors)
print(x_vectors.todense())

[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0

ما مدى نجاعة هذه المنهجية في التوسيم الآلي لأقسام الكلام؟
هيا نستكشف!


سيستعمل السطر أدناه حزمة *text_analytics* لتوليد خط ترميز أحادي موقعي لما يقارب نصف مليون كلمة، والتنبؤ بوسوم أقسام الكلام، باستعمال الانحدار اللوجيستي.


In [6]:
report = ai.pos_tagger(df, classifier="lm")
print(report)

              precision    recall  f1-score   support

         ADJ       0.89      0.85      0.87      3768
         ADP       0.93      0.97      0.95      5299
         ADV       0.90      0.81      0.85      2613
         AUX       0.95      0.98      0.97      3018
       CCONJ       0.99      0.99      0.99      1820
         DET       0.98      0.97      0.98      4637
        INTJ       0.98      0.76      0.86       156
        NOUN       0.86      0.95      0.90      9999
         NUM       0.95      0.84      0.89      1002
        PART       0.95      0.97      0.96      1339
        PRON       0.96      0.97      0.96      4561
       PROPN       0.85      0.74      0.79      3175
       PUNCT       0.99      1.00      1.00      6760
       SCONJ       0.84      0.75      0.79      1045
         SYM       0.95      0.68      0.79       116
        VERB       0.93      0.91      0.92      5992
           X       0.88      0.36      0.51       145
           _       0.98    

ثمة مناهج دائما قد تستعمل لتحسين أداء عمليات التوسيم. وهذا النشاط يبين منهجية استعمال نافذة سياق بسيطة من كلمتين لتصنيف الخصائص النحوية للكلمات.